<h1><center>Mobile App for Lottery Addiction</center></h1>

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The <a href="https://www.kaggle.com/datascienceai/lottery-dataset">data set</a> has data for 3,665 drawings, dating from 1982 to 2018 (we'll come back to this).

The scenario we're following throughout this project is fictional — the main purpose is to practice applying the concepts we learned in a setting that simulates a real-world scenario.

## Core Functions

Below, we're going to write two functions that we'll be using frequently:

* factorial() — a function that calculates factorials
* combinations() — a function that calculates combinations

In [4]:

def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def permutations(n, k):
    numerator = factorial(n)
    denominator = factorial(n-k)
    return numerator/denominator

def combinations(n,k):
    counter = factorial(n)
    denominator = factorial(n-k)*factorial(k)
    combination = counter/denominator
    return combination

## One-ticket Probability

Now focus on writing a function that calculates the probability of winning the big prize. 

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [5]:
def one_ticket_probability(user_numbers):
    total_outcomes = combinations(49, 6)
    winning_probability = (1 / total_outcomes) * 100
    print('''Your chances to win the big prize with the numbers {} are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(user_numbers,
                    winning_probability, int(total_outcomes)))

We now test a bit the function on two different outputs.

In [6]:
test_input_1 = [2, 43, 22, 23, 11, 5]
one_ticket_probability(test_input_1)

Your chances to win the big prize with the numbers [2, 43, 22, 23, 11, 5] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


## Historical Data Check for Canada Lottery

For the first version of the app users should be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

On this screen, we'll focus on exploring the historical data coming from the Canada 6/49 lottery.

In [7]:
import pandas as pd
import numpy as np

lottery = pd.read_csv('Lottery_Dataset.csv')
lottery.head(4)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34


In [8]:
lottery.shape

(3665, 11)

The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

* NUMBER DRAWN 1
* NUMBER DRAWN 2
* NUMBER DRAWN 3
* NUMBER DRAWN 4
* NUMBER DRAWN 5
* NUMBER DRAWN 6

## Function for Historical Data Check

Now write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list and serve as an input to our function.
* The engineering team wants us to write a function that prints:
    * the number of times the combination selected occurred in the Canada data set; and
    * the probability of winning the big prize in the next drawing with that combination.

Start working on writing this function:

#### 1. Extract all the winning six numbers from the historical data set as Python sets

In [10]:
# Write a function named extract_numbers() that takes as input a row of the lottery dataframe and returns a set containing all the six winning numbers:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

# Use extract_numbers() in combination with the DataFrame.apply() method to extract all the winning numbers:
winning_numbers = lottery.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

#### 2. Write a function named *check_historical_occurence()* that takes in two inputs: a Python list containing the user numbers and a pandas Series containing sets with the winning numbers (this is the Series you'll extract using the extract_numbers() function).

In [13]:
def check_historical_occurence(user_numbers, winning_numbers):
    user_numbers = set(user_numbers)
    check_matches = user_numbers == winning_numbers
    number_of_occurrence = check_matches.sum()
    
    if number_of_occurrence == 0:
        print('''The combination {} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, user_numbers))
        
    else:
        print('''The number of times combination {} has occured in the past is {}.
Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, number_of_occurrence,
                                                                            user_numbers))

Test the function with some random numbers:


In [14]:
test_numbers_1 = {4,32,12,45,28,29}
check_historical_occurence(test_numbers_1, winning_numbers )

The combination {32, 28, 29, 4, 12, 45} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {32, 28, 29, 4, 12, 45} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [16]:
test_numbers_2 = {4,5,6,7,8,9}
check_historical_occurence(test_numbers_2, winning_numbers )

The combination {4, 5, 6, 7, 8, 9} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {4, 5, 6, 7, 8, 9} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [17]:
test_numbers_3 = {33, 36, 37, 39, 8, 41}
check_historical_occurence(test_numbers_3, winning_numbers )

The number of times combination {33, 36, 37, 39, 8, 41} has occured in the past is 1.
Your chances to win the big prize in the next drawing using the combination {33, 36, 37, 39, 8, 41} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


## Multi-ticket Probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
* Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.

Let's now start writing this function.

In [33]:
def multi_ticket_probability(number_of_tickets):
    total_outcomes = combinations(49,6)
     
    winning_chance = (number_of_tickets / total_outcomes) 
    percentage_form = winning_chance * 100
    
    if number_of_tickets == 1:
        print('''Your chances to win the big prize with one ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(percentage_form, int(total_outcomes)))
    
    else:
        combinations_simplified = round(total_outcomes / number_of_tickets)   
        print('''Your chances to win the big prize with {:,} different tickets are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(number_of_tickets, percentage_form,
                                                               combinations_simplified))
    

Run a couple of test:

In [34]:
test_input = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for x in test_input:
    multi_ticket_probability(x)
    print('------------------------') # output delimiter

Your chances to win the big prize with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
------------------------
Your chances to win the big prize with 10 different tickets are 0.000072%.
In other words, you have a 1 in 1,398,382 chances to win.
------------------------
Your chances to win the big prize with 100 different tickets are 0.000715%.
In other words, you have a 1 in 139,838 chances to win.
------------------------
Your chances to win the big prize with 10,000 different tickets are 0.071511%.
In other words, you have a 1 in 1,398 chances to win.
------------------------
Your chances to win the big prize with 1,000,000 different tickets are 7.151124%.
In other words, you have a 1 in 14 chances to win.
------------------------
Your chances to win the big prize with 6,991,908 different tickets are 50.000000%.
In other words, you have a 1 in 2 chances to win.
------------------------
Your chances to win the big prize with 13,983,816 different ti

## Less Winning Numbers — Function

We're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

* Inside the app, the user inputs:
    * six different numbers from 1 to 49; and
    * an integer between 2 and 5 that represents the number of winning numbers expected
* Our function prints information about the probability of having the inputted number of winning numbers.

To help you code this function, we'll guide you through calculating the probability for having five winning numbers. For the sake of example, let's say a player chose these six numbers on a ticket: (1, 2, 3, 4 ,5 ,6). Out of these six numbers, we can form six five-number combinations:

![alt text](six.jpg "Title")

1. Write a function named *probability_less_6()* which takes in an integer between 2 and 5 and prints information about the chances of winning depending on the value of that integer.

In [37]:
def probability_less_6(number):
    total_combination = combinations(6,number)
    combinations_remaining = combinations((49 - number), (6-number))
    
    total_successful_outcomes = total_combination * combinations_remaining
    total_possible_outcomes = combinations(49, 6)
    
    winning_chance = (total_successful_outcomes / total_possible_outcomes) * 100
    combinations_simplified = round(total_possible_outcomes/total_successful_outcomes)
    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(number, winning_chance,
                                                               int(combinations_simplified)))
    


Now, let's test the function on all the three possible inputs.

In [38]:
test_input_2 = [2,3,4,5]

for x in test_input_2:
    probability_less_6(x)
    print('------------------------') # output delimiter

Your chances of having 2 winning numbers with this ticket are 19.132653%.
In other words, you have a 1 in 5 chances to win.
------------------------
Your chances of having 3 winning numbers with this ticket are 2.171081%.
In other words, you have a 1 in 46 chances to win.
------------------------
Your chances of having 4 winning numbers with this ticket are 0.106194%.
In other words, you have a 1 in 942 chances to win.
------------------------
Your chances of having 5 winning numbers with this ticket are 0.001888%.
In other words, you have a 1 in 52,969 chances to win.
------------------------
